In [52]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re

Extraer las incidencias de un partido en base a su URL en la página historiedeboca.com.ar
(Por ahora simplemente crea una lista y no funciona con tandas de penales)

In [50]:
def get_incidences(url):
    incidences = []
    
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'html.parser')
        
        incidence_table = soup.find_all('table', class_='datosPartido')
        
        if incidence_table:
            for table in incidence_table:
                rows = table.find_all('tr')
                for row in rows:
                    columns = row.find_all('td')
                    if len(columns) >= 2:
                        minute = columns[0].get_text().strip()
                        text = columns[1].get_text().strip()
                        if 'Gol' in text:
                            split_text = text.split(' de ')
                            if len(split_text) == 2:
                                incidence_type = split_text[0]
                                goalscorer = split_text[1]
                                incidences.append([minute, goalscorer, incidence_type])
                                
                        elif 'atajó un penal' in text:
                            pattern = r'^(.*?) le (.*)$'
                            coincidence = re.match(pattern, text)
                            goalkeeper = coincidence.group(1).strip()
                            incidence_type = coincidence.group(2).strip()
                            incidences.append([minute, goalkeeper, incidence_type])
                            
                        elif 'expulsado' in text:
                            pattern = r'^(.*?) \((.*?)\) (.*)$'
                            coincidence = re.match(pattern, text)
                            player = coincidence.group(1).strip() + ' (' + coincidence.group(2).strip() + ')'
                            incidence_type = coincidence.group(3).strip()
                            incidences.append([minute, player, incidence_type])
        else: incidences = ['None', 'None', 'No hubo incidencencias en el partido']
        return incidences
    else: return f'Error {response.status_code}'

In [51]:
url = 'https://www.historiadeboca.com.ar/partido/boca-3-river-0-copa-libertadores-2000/2825/2000/109/10.html'

incidences = get_incidences(url)
incidences

[["59'", 'M. Delgado', '1-0  Gol'],
 ["84'", 'J. R. Riquelme (penal)', '2-0  Gol'],
 ["86'", 'Lombardi (RP)', 'expulsado'],
 ["90'", ' M. Palermo', '3-0  Gol']]